In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from find_peak_slope import *
from find_peak_value import *
from simulate import *
from zacLib import diff,difff,awgn,mySmooth,IIRsmooth
import numba
%matplotlib Qt5

In [2]:
file = 'siga.rd'
sig,fs = rd_reader(file,1,0)
sp = fft(sig,nextpow2(sig.size,0))
sm = IIRsmooth(sp,5,0.01)
sm.tofile('test.dat')
# sm = mySmooth(sp,149,2)
# sig,sp = stairs()
# sm = np.fromfile('test.dat')

In [160]:
@numba.njit()
def get_local_maximum(x):
    return np.array([i for i in range(1,x.size-1) if x[i]>x[i+1] and x[i]>x[i-1]])

@numba.njit()
def get_local_extremum(x):
    ploc,vloc = [0 for i in range(0)],[0 for i in range(0)]
    for i in range(1,x.size-1):
        c1 = x[i]>x[i+1]
        c2 = x[i]>x[i-1]
        if c1 and c2: 
            ploc.append(i)
        if not c1 and not c2: 
            vloc.append(i)


@numba.njit()
def findBandInRange(x,peakThreshold,start,end,minBandPts=3):    
    mxi = start + x[start:end].argmax()
    i = mxi - 1
    th = x[mxi]-peakThreshold
    while i>=start and x[i] >= th:
        i -= 1
    j = mxi + 1
    while j<end and x[j]>=th:
        j += 1
    pl,vl = [0 for _ in range(0)],[0 for _ in range(0)]
    ipl,ivl = [0 for _ in range(0)],[0 for _ in range(0)]
    jpl,jvl = [0 for _ in range(0)],[0 for _ in range(0)]
    if i<start and j==end:
        return pl,vl  
    if i>=start+minBandPts:
        ipl,ivl = findBandInRange(x,peakThreshold,start,i,minBandPts)
    if i>=start and j<end:
        pl,vl = [i],[j]
    if j<end-minBandPts:
        jpl,jvl = findBandInRange(x,peakThreshold,j,end,minBandPts)   
    return ipl+pl+jpl, ivl+vl+jvl

@numba.njit()
def findBandInRange2(x,loc,peakThreshold,start,end):  
    idx = start+x[loc[start:end]].argmax()
    mxi = loc[idx]
    th = x[mxi]-peakThreshold

    i,j = mxi-1,mxi+1
    startx = loc[start-1] if start>0 else 0
    endx = loc[end+1] if end<loc.size-1 else x.size
    while i>=startx and x[i]>=th:
        i -= 1        
    while j<endx and x[j]>=th:
        j += 1

    pl,vl = [0 for _ in range(0)],[0 for _ in range(0)]
    if i<startx and j==endx:
        return pl,vl  

    id = idx-1
    while id>=start and loc[id]>i:
        id -= 1   
    jd = idx+1
    while jd<end and loc[jd]<j:
        jd += 1
    
    ipl,ivl = [0 for _ in range(0)],[0 for _ in range(0)]
    jpl,jvl = [0 for _ in range(0)],[0 for _ in range(0)]

    if id>=start:
        ipl,ivl = findBandInRange2(x,loc,peakThreshold,start,id)
    if i>=startx and j<endx:
        pl,vl = [i],[j]
    if jd<end:
        jpl,jvl = findBandInRange2(x,loc,peakThreshold,jd,end) 

    return ipl+pl+jpl, ivl+vl+jvl

@numba.njit()
def foo(x,th):
    loc = get_local_maximum(x)
    ploc1,vloc1 = findBandInRange2(x,loc,th,0,loc.size)
    return ploc1,vloc1

@numba.njit()
def bar(x,th):
    loc = get_local_maximum(x)
    sloc = loc[np.argsort(x[loc])]

In [161]:
x = sm.copy()
bar(x,0.5)
foo(x,0.5)
loc = get_local_maximum(x)
sloc = loc[np.argsort(x[loc])]
%timeit loc = get_local_maximum(x)
%timeit sloc = loc[np.argsort(x[loc])]
# %timeit bar(x,0.5)
# %timeit foo(x,0.5)

728 µs ± 1.41 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
66.4 µs ± 125 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [162]:
x = sm.copy()
th = 0.5
ploc,vloc = findBandInRange(x,th,0,x.size)
ploc1,vloc1 = foo(x,th)
%timeit findBandInRange(x,th,0,x.size)
%timeit foo(x,th)

3.56 ms ± 32.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.23 ms ± 48.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
# plt.figure()
# plt.plot(x)
# plt.plot(ploc,x[ploc],'rs')
# plt.plot(vloc,x[vloc],'rs')
# plt.plot(ploc1,x[ploc1],'g^')
# plt.plot(vloc1,x[vloc1],'gv')

In [ ]:
# foo(x,0.5)
# findBandInRange(x,0.5,0,x.size)
# %timeit findBandInRange(x,0.5,0,x.size)
# %timeit foo(x,0.5)

In [4]:
# x = np.zeros(1000)
# x[0:100] = bell(100,1)
# x[100:200] = bell(100,1)
# x[300:400] = bell(100,3)
x = sm.copy()

# plt.plot(x)

In [7]:
th = 0.1
@numba.njit()
def findBandInRange3(x,th):
    idmax,idmin = 0,0
    xmax,xmin = x[idmax],x[idmin]
    ploc,vloc = [],[]
    rising = -1
    for i in range(0,x.size):
        d = x[i] 
        if d > xmax:
            idmax = i        
            xmax = x[idmax]
            while d-x[idmin]>th:
                idmin+=1
            xmin = x[idmin]
            rising = idmin
        if d < xmin:
            idmin = i
            xmin = x[idmin]
            while x[idmax]-d>th:
                idmax+=1   
            xmax = x[idmax]
            if rising>=0:
                t = idmax-1
                while t>rising and xmax-x[t]<th:
                    t-=1
                ploc.append(t)
                vloc.append(i)
                rising = -1
    return ploc,vloc
ploc,vloc = findBandInRange3(x,th)
%timeit findBandInRange3(x,0.1)

876 µs ± 23.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [177]:
oploc,ovloc = findBandInRange(x,th,0,x.size)
plt.plot(x)
plt.plot(oploc,x[oploc],'rs')
plt.plot(ovloc,x[ovloc],'rd')
plt.plot(ploc,x[ploc],'g^')
plt.plot(vloc,x[vloc],'gv')

In [167]:
%timeit findBandInRange3(x,0.1)
%timeit findBandInRange(x,0.1,0,x.size)

826 µs ± 9.89 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
15 ms ± 15 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
# plt.figure()
# plt.plot(sp,'.',markersize=0.4)
# plt.plot(sm)
# plt.plot(ploc,sm[ploc],'r^')
# plt.plot(vloc,sm[vloc],'gv')
# plt.show()

In [ ]:
# def demo(x,sp,width,slope_th):
    # (sm,dsp,new_peak,new_pits,avg_frq,max_frq,
    #  dev_band,half_left,half_right,half_band) = find_peak_slope(sp,width,slope_th)
     
    # plot_all(sp,sm,dsp,slope_th,new_peak,new_pits,avg_frq,
    #          max_frq,dev_band,half_left,half_right,1,origin=x)

In [ ]:
# real data

# find_peak_slope_from_rd('sig0.rd',150,20,1,0)
# find_peak_slope_from_rd('sig1.rd',300,30,1,0)
# find_peak_slope_from_rd('sig2.rd',40,10,1,0)
# find_peak_slope_from_rd('sig3.rd',300,30,1,0)
# find_peak_slope_from_rd('siga.rd',300,30,1,0)

In [ ]:
# simulate data

# bell shape
# x,sp = bells(noise=0.3)
# demo(x,sp,200,20)

# square
# x,sp = squares(0.5)
# demo(x,sp,100,20)

#stair
# x,sp = stairs(0.5)
# demo(x,sp,200,30)

# random combination
# x,sp = random(noise=0.7)
# demo(x,sp,100,20)